In [2]:
!huggingface-cli download jartine/llava-v1.5-7B-GGUF llava-v1.5-7b-mmproj-f16.gguf --local-dir . --local-dir-use-symlinks False


/home/gitpod/.pyenv/versions/3.12.6/lib/python3.12/site-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
llava-v1.5-7b-mmproj-f16.gguf: 100%|█████████| 624M/624M [00:36<00:00, 17.3MB/s]
Download complete. Moving file to llava-v1.5-7b-mmproj-f16.gguf
llava-v1.5-7b-mmproj-f16.gguf


In [3]:
!huggingface-cli download jartine/llava-v1.5-7B-GGUF llava-v1.5-7b-Q4_K.gguf --local-dir . --local-dir-use-symlinks False

/home/gitpod/.pyenv/versions/3.12.6/lib/python3.12/site-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
llava-v1.5-7b-Q4_K.gguf


In [12]:
!pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.90-cp312-cp312-linux_x86_64.whl
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)


In [13]:
from llama_cpp import Llama
from llama_cpp.llama_chat_format import Llava15ChatHandler


model_file = "llava-v1.5-7b-Q4_K.gguf"
model_mmproj_file = "llava-v1.5-7b-mmproj-f16.gguf"
chat_handler = Llava15ChatHandler(clip_model_path=model_mmproj_file)
model = Llama(
        model_path=model_file,
        chat_handler=chat_handler,
        n_ctx=2048,
        n_gpu_layers=-1,  # Set to 0 if you don't have a GPU
        verbose=True,
        logits_all=True,
    )

clip_model_load: loaded meta data with 18 key-value pairs and 377 tensors from llava-v1.5-7b-mmproj-f16.gguf
clip_model_load: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
clip_model_load: - kv   0:                       general.architecture str              = clip
clip_model_load: - kv   1:                      clip.has_text_encoder bool             = false
clip_model_load: - kv   2:                    clip.has_vision_encoder bool             = true
clip_model_load: - kv   3:                   clip.has_llava_projector bool             = true
clip_model_load: - kv   4:                          general.file_type u32              = 1
clip_model_load: - kv   5:                               general.name str              = openai/clip-vit-large-patch14-336
clip_model_load: - kv   6:                        general.description str              = image encoder for LLaVA
clip_model_load: - kv   7:                     clip.vision.image_size u32              = 336

In [16]:
from PIL import Image
from io import BytesIO


def image_b64encode(img: Image) -> str:
    """ Convert image to a base64 format """
    buffered = BytesIO()
    img.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [15]:
#!pip install Pillow


  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)


In [13]:
from typing import Any
import base64
from PIL import Image
import sys
def model_inference(model: Any, request: str, image: Image) -> str:
    """ Ask model a question """
    image_b64 = image_b64encode(image)
    out_stream = model.create_chat_completion(
      messages = [
          {
              "role": "system",
              "content": "You are an assistant who perfectly describes images."
          },
          {
              "role": "user",
              "content": [
                  {"type": "image_url",
                   "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}},
                  {"type" : "text",
                   "text": request}
              ]
          }
      ],
      stream=True,
      temperature=0.2
    )

    # Get characters from stream
    output = ""
    for r in out_stream:
        data = r["choices"][0]["delta"]
        if "content" in data:
            print(data["content"], end="")
            sys.stdout.flush()
            output += data["content"]

    return output

In [17]:
img = Image.open('/workspace/chart-gpt/cat.jpg')
model_inference(model, "Please describe the image", img)


You are an assistant who perfectly describes images.USER:  describe the imageASSISTANT: 


 The image

Llama.generate: 596 prefix-match hit, remaining 1 prompt tokens to eval


 features a gray cat wearing sunglasses, giving it a cool and stylish appearance. The cat is sitting on a white surface, possibly a table, and appears to be looking at the camera. The sunglasses are placed on the cat's eyes, adding a fun and playful touch to the scene.


llama_print_timings:        load time =  142472.59 ms
llama_print_timings:      sample time =       2.69 ms /    70 runs   (    0.04 ms per token, 26061.06 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   16742.48 ms /    70 runs   (  239.18 ms per token,     4.18 tokens per second)
llama_print_timings:       total time =   16984.95 ms /    70 tokens


" The image features a gray cat wearing sunglasses, giving it a cool and stylish appearance. The cat is sitting on a white surface, possibly a table, and appears to be looking at the camera. The sunglasses are placed on the cat's eyes, adding a fun and playful touch to the scene."

In [9]:
from transformers import VitsModel, VitsTokenizer
import torch
model = VitsModel.from_pretrained("facebook/mms-tts-eng")
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-eng")

text = "Hello World"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    data = model(**inputs).waveform.cpu().numpy()  

ImportError: 
VitsModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [8]:
#!pip install torch


In [10]:
from io import BytesIO
import scipy
import numpy as np 
sample_rate = 16000 
buffer = BytesIO()
data_int16 = (data * np.iinfo(np.int16).max).astype(np.int16)
scipy.io.wavfile.write(buffer, rate=sample_rate, data=data_int16.squeeze())
data_wav = buffer.getbuffer().tobytes()

ModuleNotFoundError: No module named 'scipy'

In [ ]:
import streamlit as st


# LLaVA Model
model_mmproj_file = "llava-v1.5-7b-mmproj-f16.gguf"
model_file = "llava-v1.5-7b-Q4_K.gguf"

@st.cache_resource
def load_chat_handler():
    """ Load LLAVA chat handler """
    return Llava15ChatHandler(clip_model_path=model_mmproj_file)

@st.cache_resource
def load_model():
    """ Load model """
    chat_handler = load_chat_handler()
    return Llama(
        model_path=model_file,
        chat_handler=chat_handler,
        n_ctx=2048,
        n_gpu_layers=-1,  # Set to 0 if you don't have a GPU,
        verbose=True,
        logits_all=True,
    )

In [ ]:
def st_describe(model: Any, tts: Any, prompt: str, image: Image) -> str:
    """ Describe image with a prompt in browser """
    with st.spinner('Describing the image...'):
        response = model_inference(model, prompt, image)
    st_generate_audio(tts, response)

def st_generate_audio(tts: Any, text: str):
    """ Generate and play the audio"""
    with st.spinner('Generating the audio...'):
        wav_data = tts.generate_audio(text)    
    st_autoplay(wav_data)

def st_autoplay(wav: bytes):
    """ Create an audio control in browser """
    b64 = base64.b64encode(wav).decode()
    md = f"""
         <audio controls autoplay="true">
         <source src="data:audio/mp3;base64,{b64}" type="audio/wav">
         </audio>
         """
    st.markdown(md, unsafe_allow_html=True)

In [ ]:
def main():
    """ Main app """
    st.title('LLAVA AI Assistant')

    with st.spinner('Loading the models, please wait'):
        model = load_model()
        tts = load_tts_model()

    img_file_buffer = st.camera_input("Take a picture")
    if img_file_buffer:
        cam_image = Image.open(img_file_buffer)
        if st.button('Describe the image'):
            st_describe(model, tts, "Please describe the image.", cam_image)
        if st.button('Read the label'):
            st_describe(model, tts, "Read the text on the image. If there is no text, write that the text cannot be found.", cam_image)